In [ ]:
from google.colab import drive
import zipfile

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define paths
zip_path = '/content/drive/MyDrive/TeethDataSet.zip'  # Path to the zip file
extract_path = '/content/drive/MyDrive/TeethDataSet'  # Path where files will be extracted

# Step 3: Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Step 4: Verify extraction
!ls /content/drive/MyDrive/TeethDataSet


Mounted at /content/drive
Teeth_Dataset


In [ ]:
import os
from PIL import Image, ImageOps
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import random

# Custom transformation functions
def resize(image, size):
    return image.resize(size, Image.ANTIALIAS)

def random_horizontal_flip(image, p=0.5):
    if random.random() < p:
        return ImageOps.mirror(image)
    return image

def random_rotation(image, degrees):
    return image.rotate(random.uniform(-degrees, degrees))

def color_jitter(image, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2):
    # Adjust brightness
    brightness_factor = random.uniform(max(0, 1 - brightness), 1 + brightness)
    image = ImageEnhance.Brightness(image).enhance(brightness_factor)
    # Adjust contrast
    contrast_factor = random.uniform(max(0, 1 - contrast), 1 + contrast)
    image = ImageEnhance.Contrast(image).enhance(contrast_factor)
    # Adjust saturation
    saturation_factor = random.uniform(max(0, 1 - saturation), 1 + saturation)
    image = ImageEnhance.Color(image).enhance(saturation_factor)
    # Adjust hue
    hue_factor = random.uniform(-hue, hue)
    image = ImageOps.colorize(image.convert('L'), (0, 0, 0), (int(hue_factor*255), int(hue_factor*255), int(hue_factor*255)))
    return image

def normalize(image, mean, std):
    image = np.array(image).astype(np.float32)
    image = (image / 255.0 - mean) / std
    return torch.tensor(image).permute(2, 0, 1)

# Custom Dataset class
class TeethDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_paths = []
        self.labels = []
        self.transform = transform
        self.class_to_idx = self._find_classes(root_dir)
        self._make_dataset()

    def _find_classes(self, dir):
        classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        classes.sort()
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return class_to_idx

    def _make_dataset(self):
        for cls_name, cls_idx in self.class_to_idx.items():
            cls_dir = os.path.join(self.root_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                if img_name.endswith(('png', 'jpg', 'jpeg')):
                    img_path = os.path.join(cls_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(cls_idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Custom transform function combining all transformations
def custom_transform(image):
    image = resize(image, (224, 224))
    image = random_horizontal_flip(image)
    image = random_rotation(image, 10)
    image = color_jitter(image)
    image = normalize(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    return image

# Instantiate the datasets
train_dataset = TeethDataset(root_dir='/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Training', transform=custom_transform)
val_dataset = TeethDataset(root_dir='/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Validation', transform=custom_transform)
test_dataset = TeethDataset(root_dir='/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Testing', transform=lambda x: normalize(resize(x, (224, 224)), mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(SimpleCNN, self).__init__()
        # Define the layers of the CNN
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)  # Adjust input size based on image dimensions
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)  # Flatten the tensor for the fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = SimpleCNN(num_classes=7)


In [ ]:
import torch.optim as optim

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Suitable for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate of 0.001


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    model.to(device)  # Ensure model is on the correct device
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to the correct device
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

        # Validate the model
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)  # Move data to the correct device
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        accuracy = correct / total
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')


In [ ]:
def evaluate_model(model, test_loader):
    model.to(device)  # Ensure model is on the correct device
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to the correct device
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=100352, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=7, bias=True)
)

In [ ]:
from PIL import Image, ImageEnhance, ImageOps
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import random

# Custom transformation functions
def resize(image, size):
    return image.resize(size, Image.LANCZOS)

def random_horizontal_flip(image, p=0.5):
    if random.random() < p:
        return ImageOps.mirror(image)
    return image

def random_rotation(image, degrees):
    return image.rotate(random.uniform(-degrees, degrees))

def color_jitter(image, brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2):
    # Adjust brightness
    brightness_factor = random.uniform(max(0, 1 - brightness), 1 + brightness)
    image = ImageEnhance.Brightness(image).enhance(brightness_factor)
    # Adjust contrast
    contrast_factor = random.uniform(max(0, 1 - contrast), 1 + contrast)
    image = ImageEnhance.Contrast(image).enhance(contrast_factor)
    # Adjust saturation
    saturation_factor = random.uniform(max(0, 1 - saturation), 1 + saturation)
    image = ImageEnhance.Color(image).enhance(saturation_factor)
    # Adjust hue
    hue_factor = random.uniform(-hue, hue)
    image = ImageOps.colorize(image.convert('L'), (0, 0, 0), (int(hue_factor*255), int(hue_factor*255), int(hue_factor*255)))
    return image

def normalize(image, mean, std):
    image = np.array(image).astype(np.float32)
    image = (image / 255.0 - mean) / std
    return torch.tensor(image).permute(2, 0, 1)

# Custom Dataset class
class TeethDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_paths = []
        self.labels = []
        self.transform = transform
        self.class_to_idx = self._find_classes(root_dir)
        self._make_dataset()

    def _find_classes(self, dir):
        classes = [d.name for d in os.scandir(dir) if d.is_dir()]
        classes.sort()
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return class_to_idx

    def _make_dataset(self):
        for cls_name, cls_idx in self.class_to_idx.items():
            cls_dir = os.path.join(self.root_dir, cls_name)
            for img_name in os.listdir(cls_dir):
                if img_name.endswith(('png', 'jpg', 'jpeg')):
                    img_path = os.path.join(cls_dir, img_name)
                    self.image_paths.append(img_path)
                    self.labels.append(cls_idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Custom transform function combining all transformations
def custom_transform(image):
    image = resize(image, (224, 224))
    image = random_horizontal_flip(image)
    image = random_rotation(image, 10)
    image = color_jitter(image)
    image = normalize(image, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    return image

# Instantiate the datasets
train_dataset = TeethDataset(root_dir='/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Training', transform=custom_transform)
val_dataset = TeethDataset(root_dir='/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Validation', transform=custom_transform)
test_dataset = TeethDataset(root_dir='/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Testing', transform=lambda x: normalize(resize(x, (224, 224)), mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
def normalize(image, mean, std):
    image = np.array(image).astype(np.float32)  # Ensure the image is float32
    image = (image / 255.0 - mean) / std
    return torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)  # Ensure tensor is float32


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device, dtype=torch.float32), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

        # Validate the model
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device, dtype=torch.float32), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        accuracy = correct / total
        print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')


In [ ]:
def evaluate_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device, dtype=torch.float32), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    print(f'Test Accuracy: {accuracy:.4f}')


In [ ]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

# Evaluate the model
evaluate_model(model, test_loader)


Epoch 1/10, Loss: 2.0464
Validation Loss: 1.9348, Accuracy: 0.1751
Epoch 2/10, Loss: 1.9371
Validation Loss: 1.9333, Accuracy: 0.1722
Epoch 3/10, Loss: 1.9326
Validation Loss: 1.9355, Accuracy: 0.1751
Epoch 4/10, Loss: 1.9329
Validation Loss: 1.9303, Accuracy: 0.1751
Epoch 5/10, Loss: 1.9324
Validation Loss: 1.9447, Accuracy: 0.1751
Epoch 6/10, Loss: 1.9338
Validation Loss: 1.9313, Accuracy: 0.1751
Epoch 7/10, Loss: 1.9327
Validation Loss: 1.9333, Accuracy: 0.1751
Epoch 8/10, Loss: 1.9367
Validation Loss: 1.9312, Accuracy: 0.1751
Epoch 9/10, Loss: 1.9336
Validation Loss: 1.9310, Accuracy: 0.1751
Epoch 10/10, Loss: 1.9310
Validation Loss: 1.9298, Accuracy: 0.1751
Test Accuracy: 0.1340


In [ ]:
from PIL import Image

def predict_image(model, image_path, transform):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Move the image to the same device as the model (e.g., GPU if available)
    image = image.to(device)
    model = model.to(device)

    # Set the model to evaluation mode
    model.eval()

    # Predict the class
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)

    return predicted.item()

# Example usage:
image_path = '/content/drive/MyDrive/TeethDataSet/Teeth_Dataset/Testing/MC/mc_1202.jpg'  # Replace with your image path
predicted_class = predict_image(model, image_path, transform=custom_transform)
print(f'Predicted class: {predicted_class}')


Predicted class: 5


In [ ]:
from collections import Counter

train_labels = [label for _, label in train_dataset]
class_distribution = Counter(train_labels)
print("Class distribution in training data:", class_distribution)


Class distribution in training data: Counter({3: 540, 5: 540, 0: 480, 1: 450, 6: 393, 2: 360, 4: 324})


In [ ]:
# Reverse the class_to_idx dictionary
idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}

# Get the class name for the predicted class (5 in this case)
predicted_class_name = idx_to_class[5]
print(f'Class 5 corresponds to: {predicted_class_name}')


Class 5 corresponds to: OLP
